In [ ]:
import pandas as pd
import numpy as np
import os
from tabula import read_pdf
import re
from dateutil import parser
from datetime import datetime
from datetime import date

In [ ]:
def process_text(file_number):

# DEFINE BOX TO SCAN THEN EXTRACT DATA
    
    box = [212.00, 325.00, 300.00, 600.00]
    extract_data = read_pdf(directory + '/DL' + file_number + '.pdf', pages='1', area=[box], stream=True)

# DEFINE WHICH PART OF TEXT TO EXTRACT

    string_extract = ''.join(map(str, extract_data))
    diperiksa_find = '(diperiksa?\s*:\s*[0-9.,]+)'
    diperiksa_data = re.findall(diperiksa_find, string_extract)
    diperiksa_data = diperiksa_data[0].replace(',', '').replace('.','').replace('diperiksa','').replace(':','').strip()
    diperiksa_data = int(diperiksa_data)
    
    positif_find = '(Positif\sCOVID-19?\s*:\s*[0-9.,]+)'
    positif_data = re.findall(positif_find, string_extract)
    positif_data = positif_data[0].replace(',', '').replace('.','').replace('Positif COVID-19','').replace(':','').strip()
    positif_data = int(positif_data)
    
    sembuh_find = '(Sembuh\s\\(Positif\sCOVID-19\\)?\s*:\s*[0-9.,]+)'
    sembuh_data = re.findall(sembuh_find, string_extract)
    sembuh_data = sembuh_data[0].replace(',', '').replace('.','').replace('Sembuh (Positif COVID-19)','').replace(':','').strip()
    sembuh_data = int(sembuh_data)
    
    meninggal_find = '(Meninggal\s\\(Positif\sCOVID-19\\)?\s*:\s*[0-9.,]+)'
    meninggal_data = re.findall(meninggal_find, string_extract)
    meninggal_data = meninggal_data[0].replace(',', '').replace('.','').replace('\'','').replace('Meninggal (Positif COVID-19)','').replace(':','').strip()
    meninggal_data = int(meninggal_data)
    
# ADD SOME INDICES AND DATE

    pr_kum = 100 * positif_data / diperiksa_data
    cfr_kum = 100 * meninggal_data / positif_data
    
    tgl = file_number[:2] + "-" + file_number[2:4] + "-" + file_number[4:]
    tgl = datetime.strptime(tgl, '%d-%m-%y')
    tgl = tgl.date()

# PUT THE EXTRACTED DATA INTO A DATAFRAME

    data = {'DPRIKS': [diperiksa_data], 'PSTF KUM': [positif_data], 'SMBH KUM': [sembuh_data], 'MNGL KUM':[meninggal_data], 
            'PR KUM': [pr_kum], 'CFR KUM': [cfr_kum], 'TGL': [tgl]}
    columns = ['DPRIKS', 'PSTF KUM', 'SMBH KUM', 'MNGL KUM', 'PR KUM', 'CFR KUM', 'TGL']
    
    df_err_total = pd.DataFrame(data=data, columns=columns)
    
    return df_err_total


In [ ]:
# month_name = 'Maret'
# directory = 'ErrorPDF/' + month_name
# file_number = '190320'

In [ ]:
# FUNCTION TO INITIATE EXTRACTION PROCESS

def process_all(file_number):
    df_err_total = process_text(file_number)
    return df_err_total

In [ ]:
error_files = []
error_msgs = []

total_columns = ['DPRIKS', 'PSTF KUM', 'SMBH KUM', 'MNGL KUM', 'PR KUM', 'CFR KUM', 'TGL']

agg_err_total = pd.DataFrame(columns=total_columns)

# IMPLEMENT FUNCTION WITH ITERATION OF FILES IN FOLDER

month_name = 'Agustus'
directory = 'ErrorPDF/' + month_name

for file in os.listdir(directory):
    file_number = file[2:8]
    print('Processing ' + file_number)
    try:
        df_err_total = process_all(file_number)
        agg_err_total = agg_err_total.append(df_err_total, ignore_index=True)
    except Exception as e:
        error_files.append(file_number)
        error_msgs.append(e)
        print(e)
        continue
        
# HANDLE LOG OF ERRORS

error_dict = {'Error Files': error_files, 'Error Messages': error_msgs}
df_error = pd.DataFrame(data=error_dict)
df_error.to_excel(directory + '/ErrorTotalLog_' + month_name + '.xlsx')

# WRITE AGGREGATED DATA TO EXCEL

agg_err_total.to_excel(directory + '/AggErrTTL_' + month_name + '.xlsx')


# THIS CODE FAILED WITH THE ERROR 'LIST INDEX OUT OF RANGE' PROBABLY THERE IS SOMETHING WRONG IN THE OCR PROCESS.
# I DECIDE TO GO FULLY MANUAL INSTEAD FOR THE ERROR PDF